In [1]:
import re
from datetime import datetime
from datetime import timedelta
from pymongo import MongoClient 
from textblob import TextBlob

In [2]:
# Funzione per filtrare gli hashtag, mantendendo così solo quelli contenenti testo e numeri
def filterHashtag(text):
    return re.match(r"(([a-zA-Z]+\d+)+)", text)

In [3]:
# Funzione per la pulizia del testo dei tweet, così da renderli analizzabili dalla libreria TextBlob per la sentiment analysis
def clean_tweet(tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [4]:
# Connessione al database MongoDB
myclient = MongoClient("mongodb://localhost:27017/") 
db = myclient["ProgettoData"]
tweetCol = db["Twitter"]
coinDataCol = db["Coingecko"]
coinTweetDataCol = db["CoinTweetData"]

In [5]:
# Variabile per specificare di quanti giorni andare indietro per effettuare l'integrazione dei dati (es. 1 = ieri)
backDays = 1
startDate = datetime.timestamp((datetime.now()- timedelta(backDays)).replace(hour=0, minute=0, second=0, microsecond=0))
endDate = datetime.timestamp((datetime.now()- timedelta(backDays)).replace(hour=23, minute=59, second=59, microsecond=0))

loopDate = startDate

# Time slice da 5 minuti
minutesAgg = 5*60
        
doc = {"day": datetime.fromtimestamp(startDate), "data": []}
        
while loopDate < endDate:
    loopDate = loopDate + minutesAgg
    coinData = coinDataCol.find({"dogecoin.last_updated_at": {"$lt": loopDate, "$gte": (loopDate - minutesAgg)}})
    tweetData = tweetCol.find({"timestamp_ms": {"$lt": (loopDate*1000), "$gte": ((loopDate - minutesAgg)*1000)}})
    
    price, marketCap, volume, numTweetTot, numTweet, numRetweet, text, posTweets, negTweets, neutTweets = 0, 0, 0, tweetData.count(), 0, 0, [], 0, 0, 0
    
    if coinData.count() != 0:
        for x in coinData:
            price += x["dogecoin"]["eur"]
            marketCap += x["dogecoin"]["eur_market_cap"]
            volume += x["dogecoin"]["eur_24h_vol"]
        price /= coinData.count()
        marketCap /= coinData.count()
        volume /= coinData.count()
    
    if numTweetTot != 0:
        for x in tweetData:
            if x["is_retweet"]:
                numRetweet += 1
            else:
                numTweet += 1
            if "hashtags" in x:
                for hashtag in x["hashtags"]:
                    if filterHashtag(hashtag):
                        text.append(hashtag)
            # sentiment analysis
            analysis = TextBlob(clean_tweet(x["text"]))
            if analysis.sentiment.polarity > 0:
                posTweets += 1
            else:
                if analysis.sentiment.polarity == 0:
                    neutTweets += 1
                else:
                    negTweets += 1
    
    timesliceData = {"date": datetime.fromtimestamp(loopDate), "price": price, "market_cap": marketCap, "volume": volume, "number_of_tweet_tot": numTweetTot, "number_of_tweet": numTweet, "number_of_retweet": numRetweet, "hashtags": text, "posTweets": posTweets, "negTweets": negTweets, "neutTweets": neutTweets}
    doc["data"].append(timesliceData)

coinTweetDataCol.insert_one(doc)    

<ipython-input-5-ed651d08a84c>:15: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  price, marketCap, volume, numTweetTot, numTweet, numRetweet, text, posTweets, negTweets, neutTweets = 0, 0, 0, tweetData.count(), 0, 0, [], 0, 0, 0
<ipython-input-5-ed651d08a84c>:17: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if coinData.count() != 0:
<ipython-input-5-ed651d08a84c>:22: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  price /= coinData.count()
<ipython-input-5-ed651d08a84c>:23: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  marketCap /= coinData.count()
<ipython-input-5-ed651d08a84c>:24: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  volume /= coinData.count()
